Introducing manual compression of image captions on stale (offline) data

In [1]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
from transformers import AdamW
from datasets import load_dataset
import torch
import torch.optim as optim
from collections import Counter
import fiftyone
import torch.nn as nn
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np
import os
import time
import glob

/Users/sagnik/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the pre-trained model and its components
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

/Users/sagnik/Library/Python/3.9/lib/python/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [3]:
# Load a dataset (for example, a subset of the COCO dataset)
# TODO: Potential datasets with repititive nature that can be used: MS COCO, Flickr30k, Visual Genome, SBU Captions 

# load small part of the coco dataset from all the .jpg images in datasets/mscoco/test2015
dataset = load_dataset("datasets/mscoco/test2015/", split="test[:10]")

Resolving data files: 100%|██████████| 81434/81434 [00:00<00:00, 1152607.01it/s]


In [4]:
def generate_caption(image, max_length=128):
    inputs = feature_extractor(images=image, return_tensors="pt")
    output_ids = model.generate(inputs["pixel_values"], max_length=max_length)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption

In [5]:
# Iterate over the dataset and generate captions
max_length = 128
generated_captions = []

for data in dataset:
    image = data['image']
    caption = generate_caption(image, max_length=max_length)
    generated_captions.append(caption)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


In [ ]:
def update_encoding_dict(captions, encoding_dict):
    for caption in captions:
        words = caption.split()
        encoding_dict.update(words) # purpose of update is to add the words to the dictionary if they don't exist
    return encoding_dict

In [ ]:
encoding_dict = Counter() # Counter is a subclass of dictionary for counting hashable objects
threshold = 2 # threshold for word frequency # TODO: find a good threshold

update_encoding_dict(generated_captions, encoding_dict)

# Optionally, create a more compressed form based on frequency
compressed_dict = {word: idx for idx, (word, freq) in enumerate(encoding_dict.items()) if freq > threshold}

In [6]:
class CaptionAutoencoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, max_seq_length):
        super(CaptionAutoencoder, self).__init__()

        # Encoder
        self.encoder_embedding = nn.Embedding(vocab_size, embedding_dim) # input shape has to be (batch_size, sequence_length), output shape is (batch_size, sequence_length, embedding_dim)
        self.encoder_rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True) # output shape is (batch_size, sequence_length, hidden_dim)
        self.max_seq_length = max_seq_length

        # Decoder
        self.decoder_rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True) # output shape is (batch_size, sequence_length, hidden_dim)
        self.decoder_output = nn.Linear(hidden_dim, vocab_size)

    def encode(self, captions):
        embedded = self.encoder_embedding(captions)
        encoded, _ = self.encoder_rnn(embedded)
        return encoded[:, -1, :]

    def decode(self, encoded):
        # Repeat the encoded state across the sequence length
        repeated_encoded = encoded.unsqueeze(1).repeat(1, self.max_seq_length, 1) 
        decoded, _ = self.decoder_rnn(repeated_encoded)
        output = self.decoder_output(decoded)
        return output

    def forward(self, captions):
        encoded = self.encode(captions)
        decoded = self.decode(encoded)
        return decoded


In [7]:
class CaptionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, captions):
        self.encodings = encodings
        self.captions = captions

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.captions[idx])
        return item

    def __len__(self):
        return len(self.captions)

In [8]:
# Assuming `dataset` is your dataset containing images and captions
images = [data['image'] for data in dataset]
captions = generated_captions

# Process images and captions
inputs = feature_extractor(images=images, return_tensors="pt")
outputs = tokenizer(captions, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Assuming 'captions' is a tensor of tokenized captions generated by VLM
vocab_size = tokenizer.vocab_size
embedding_dim = 50257
hidden_dim = 512
max_seq_length = 128
autoencoder = CaptionAutoencoder(vocab_size, embedding_dim, hidden_dim, max_seq_length)
autoencoder_output = autoencoder(outputs["input_ids"])

# Create dataset and dataloader
train_dataset = CaptionDataset(inputs, outputs["input_ids"])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [9]:
class LoRALayer(nn.Module):
    def __init__(self, original_weight, rank):
        super(LoRALayer, self).__init__()
        self.original_weight = original_weight
        self.rank = rank
        self.U = nn.Parameter(torch.Tensor(self.original_weight.size(0), self.rank))
        self.V = nn.Parameter(torch.Tensor(self.rank, self.original_weight.size(1)))
        nn.init.xavier_uniform_(self.U)
        nn.init.xavier_uniform_(self.V)

    def forward(self):
        return self.original_weight + self.U @ self.V

In [10]:
# Modify the first attention layer of the encoder
# TODO: Try modifying other layers as well and check the results
lora_layers = []

with torch.no_grad():
    original_weight = model.encoder.encoder.layer[0].attention.output.dense.weight
    lora_layer = LoRALayer(original_weight, rank=10).forward()  # Choose an appropriate rank
    # assign the new layer to the model
    model.encoder.encoder.layer[0].attention.output.dense.weight.copy_(lora_layer)
    # add the layer of the model to the list of LoRA layers
    lora_layers.append(model.encoder.encoder.layer[0].attention.output.dense)

In [11]:
def is_lora_param(param, lora_layer):
    # check if the parameter is part of the LoRA layer
    print (lora_layer.parameters())
    print ("nuj")
    print (param)
    return param in lora_layer.parameters()

In [19]:
def custom_loss(outputs, batch, lora_layers, autoencoder, standard_lambda_val = 1, lora_lambda_val = 0.01, compression_lambda_val = 0.01):
    # Standard captioning loss
    standard_loss = outputs.loss

    # Autoencoder compression reward
    captions = batch['labels']
    compressed_captions = autoencoder.encode(captions)
    # Measure the sparsity of the compressed representation (e.g., using L1 norm) # TODO: Try other measures
    compression_reward = torch.norm(compressed_captions, p=1)
    # Adjust the reward: lower norm (more sparse) should lead to lower loss (higher reward)
    compression_loss = -compression_reward

    # Optionally, add a term for LoRA regularization if needed
    lora_regularization = 0
    # for param in model.parameters():
    #     for lora_layer in lora_layers:
    #         if is_lora_param(param, lora_layer):
    #             lora_regularization += torch.norm(param)
    return standard_lambda_val* standard_loss + compression_lambda_val * compression_loss + lora_lambda_val * lora_regularization

In [24]:
ae_criterion1 = nn.CrossEntropyLoss()

def ae_criterion2 (reconstructed_caption, original_caption, end_of_text_token_id):

    loss= 0

    for i in range(len(original_caption)):

        # remove all end of text tokens from the right in original caption
        trim_index = 0
        for j in range(len(original_caption[i])-1, -1, -1):
            if original_caption[i][j] != end_of_text_token_id:
                trim_index = j
                break
        trim_index += 1

        # Trim the trailing spaces
        trimmed_original = original_caption[i][:trim_index]
        trimmed_reconstructed = reconstructed_caption[i][:trim_index]

        # Calculate the loss (assuming cross-entropy loss)
        loss += ae_criterion1(trimmed_reconstructed, trimmed_original)

    return loss

In [25]:
# Fine tuning using custom loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

vlm_lr = 5e-5 # TODO: find a good learning rate
ae_lr = 1e-3 # TODO: find a good learning rate
num_epochs = 50 # TODO: find a good number of epochs

vlm_optimizer = AdamW([param for param in model.parameters() if param.requires_grad], lr=vlm_lr)
ae_optimizer = optim.Adam(autoencoder.parameters(), lr=ae_lr) 

for epoch in range(num_epochs):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Fine tune VLM with custom loss
        # Forward pass
        model.zero_grad()
        outputs = model(**batch)
        vlm_loss = custom_loss(outputs, batch, lora_layers, autoencoder)
        # Backward pass and optimization
        vlm_optimizer.zero_grad()
        vlm_loss.backward()
        vlm_optimizer.step()

        # Train the autoencoder
        autoencoder.zero_grad()
        captions = batch['labels']
        compressed_captions = autoencoder.encode(captions)
        reconstructed_captions = autoencoder.decode(compressed_captions)
        # reconstructed_flat = reconstructed_captions.view(-1, reconstructed_captions.size(-1))
        # captions_flat = captions.view(-1)
        end_of_text_token_id = tokenizer.encode('<|endoftext|>')[0]
        ae_loss = ae_criterion2(reconstructed_captions, captions, end_of_text_token_id)
        ae_loss.backward()
        ae_optimizer.step()

        # TODO: change loss as combination of vlm_loss and ae_loss instead of individual losses

        # Update progress bar
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(vlm_loss=vlm_loss.item(), ae_loss=ae_loss.item())

  0%|          | 0/1 [00:00<?, ?it/s]/var/folders/wd/7s_rgclx5rlc79rrjnspznh00000gn/T/ipykernel_46545/3431795457.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/var/folders/wd/7s_rgclx5rlc79rrjnspznh00000gn/T/ipykernel_46545/3431795457.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.captions[idx])
Epoch 49: 100%|██████████| 1/1 [00:42<00:00, 42.34s/it, ae_loss=21.6, vlm_loss=-51.2]


In [26]:
# create directory to save the model if it doesn't exist
if not os.path.exists("models_auto_compress_online_data"):
    os.mkdir("models_auto_compress_online_data")
# save model checkpoint to models directory using current timestamp and date
torch.save(model.state_dict(), f"models_auto_compress_online_data/{time.strftime('%Y%m%d-%H%M%S')}.pth")


In [27]:
# load latest model checkpoint among all the saved models
latest_model = torch.load(max(glob.glob('models_auto_compress_online_data/*.pth'), key=os.path.getctime))
# load the model with the latest checkpoint
model.load_state_dict(latest_model)

<All keys matched successfully>

In [32]:
# Generate captions for the test dataset
generated_captions_custom_model = []
generated_captions_custom_model_pre_compression = []
# Iterate over the dataset and generate captions
for data in dataset:
    image = data['image']
    # use autoencoder to encode and decode the caption
    caption = generate_caption(image)
    generated_captions_custom_model_pre_compression.append(caption)
    caption = tokenizer(caption, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    caption = caption['input_ids']
    caption = caption.to(device)
    print (caption)
    compressed_caption = autoencoder.encode(caption)
    compressed_caption = compressed_caption.to(device)
    # print (compressed_caption)
    reconstructed_caption = autoencoder.decode(compressed_caption)
    reconstructed_caption = reconstructed_caption.to(device)
    reconstructed_caption = reconstructed_caption.cpu()
    reconstructed_caption = reconstructed_caption.detach().numpy()
    reconstructed_caption = np.argmax(reconstructed_caption, axis=2)
    print (reconstructed_caption)
    reconstructed_caption = tokenizer.decode(reconstructed_caption[0], skip_special_tokens=True)
    generated_captions_custom_model.append(reconstructed_caption)

tensor([[   64,  4077,  7779, 19584,  1306,   284,   257, 20799,   220, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 5

In [ ]:
# Encode compressed dictionary word using manual huffman encoding

In [ ]:
# Replace compressed_dict words occurring in the generated_captions_custom_model with their corresponding huffman encoding

In [ ]:
# compare encoded generated_captions_custom_model + huffman encoding dictionary information with the original generated_captions to calculate compression ratio

In [33]:
# print generated_captions and generated_captions_custom_model elementwise to compare the results
for i in range(len(generated_captions)-1):
    print (generated_captions[i], generated_captions_custom_model_pre_compression[i], generated_captions_custom_model[i])

a green truck parked next to a curb  a green truck parked next to a curb  a green green parked a a a                                                                                                                         
a man is walking down the street with a skateboard  a man is walking down the street with a skateboard  a man is is a a a a a a                                                                                                                      
a baseball player swinging a bat at a ball  a baseball player swinging a bat at a ball  a man player a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
a cow is standing in a field of grass  a cow is standing in a field of grass  a man is standing a a a a                                                                              

improve autoencoder architecture, use better semantic meaning preserving metric instead of simply cross entropy